## Импорт библиотек

---

In [ ]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import cv2

from ultralytics import YOLO

## Обучение модели

---

### Загрузка предобученной модели

---

[Модели доступные для transfer learning](https://docs.ultralytics.com/models/)

In [ ]:
model = YOLO('yolov8n.pt')

### Обучение модели

---

Настраиваемые параметры (engine/trainer) для обучения модели:
- task=detect, 
- mode=train,
- model=yolov8n.pt, 
- data=coco.yaml, 
- epochs=30, 
- time=None, 
- patience=100, 
- batch=16, 
- imgsz=640, 
- save=True, 
- save_period=-1, 
- cache=False, 
- device=None, 
- workers=8, 
- project=None, 
- name=train6, 
- exist_ok=False, 
- pretrained=True, 
- optimizer=auto, 
- verbose=True, 
- seed=0, 
- deterministic=True, 
- single_cls=False, 
- rect=False, 
- cos_lr=False, 
- close_mosaic=10, 
- resume=False, 
- amp=True, 
- fraction=1.0, 
- profile=False, 
- freeze=None, 
- multi_scale=False, 
- overlap_mask=True, 
- mask_ratio=4, 
- dropout=0.0, 
- val=True, 
- split=val, 
- save_json=False, 
- save_hybrid=False, 
- conf=None, 
- iou=0.7, 
- max_det=300, 
- half=False, 
- dnn=False, 
- plots=True, 
- source=None, 
- vid_stride=1, 
- stream_buffer=False, 
- visualize=False, 
- augment=False, 
- agnostic_nms=False, 
- classes=None, 
- retina_masks=False, 
- embed=None, 
- show=False, 
- save_frames=False, 
- save_txt=False, 
- save_conf=False, 
- save_crop=False, 
- show_labels=True, 
- show_conf=True, 
- show_boxes=True, 
- line_width=None, 
- format=torchscript, 
- keras=False, 
- optimize=False, 
- int8=False, 
- dynamic=False, 
- simplify=False, 
- opset=None, 
- workspace=4, 
- nms=False, 
- lr0=0.01, 
- lrf=0.01, 
- momentum=0.937, 
- weight_decay=0.0005, 
- warmup_epochs=3.0, 
- warmup_momentum=0.8, 
- warmup_bias_lr=0.1, 
- box=7.5, 
- cls=0.5, 
- dfl=1.5, 
- pose=12.0, 
- kobj=1.0, 
- label_smoothing=0.0, 
- nbs=64, 
- hsv_h=0.015, 
- hsv_s=0.7, 
- hsv_v=0.4, 
- degrees=0.0, 
- translate=0.1, 
- scale=0.5, 
- shear=0.0, 
- perspective=0.0, 
- flipud=0.0, 
- fliplr=0.5, 
- bgr=0.0, 
- mosaic=1.0, 
- mixup=0.0, 
- copy_paste=0.0, 
- auto_augment=randaugment, 
- erasing=0.4, 
- crop_fraction=1.0, 
- cfg=None, 
- tracker=botsort.yaml, 
- save_dir=runs/detect/train6

In [ ]:
results = model.train(
    data='data/data.yaml',

    imgsz=640,

    epochs=300, 
    batch=16, 
    workers=0,
    seed=42,
    optimize=True,
    single_cls=True,
    dropout=0.2,
    visualize=True,
    augment=True,
    iou=0.92,
    lr0=0.0025, # 0.001
    verbose=True,
    mask_ratio=14,
    optimizer='Adam', 
    # device='mps',


    # После обучения модель сохранится в папку 'runs/train/train', 
    # которую нужно в дальнейшем указывать в качестве параметра ниже

    # name='train17',
    # exist_ok=True
)

In [ ]:
model.save('yolov8n_30_epoch.pt')

## Оценка модели

---

### Загрузка модели

---


In [ ]:
model = YOLO('/Users/mac/dev/gov/Real-Time-Guns-Detection/runs/detect/train17/weights/last.pt')

### Проверка на изображениях

---


In [ ]:
res = model.predict(
    Path(os.path.join('./data', 'test', 'images'))
)

**Легко:**

In [ ]:
for idx in [10, 13, 14, 20]:
    res[idx].show()

**Сложно:**

In [ ]:
for idx in [90, 330, 550, 555]:
    res[idx].show()

**Часть обнаружил, но на идентичном фото не обнаружил:**

In [ ]:
for idx in [200, 210, 230, 420, 453, 455]:
    res[idx].show()

**Запомнил место:**

In [ ]:
for idx in [557, 558]:
    res[idx].show()

#### Проверка на одном изображении

---

In [ ]:
res_1 = model.predict('/Users/mac/Downloads/2024-04-24 21.23.52.jpg')
res_1[0].show()

### Проверка на видео

---

In [ ]:
video_path = "/Volumes/NO NAME/VIDEO/output100.avi"

cap = cv2.VideoCapture(video_path)
frame_number = 0

os.makedirs("images", exist_ok=True)
os.makedirs("labels", exist_ok=True)


while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True)
        annotated_frame = results[0].plot()
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

cap.release()
cv2.destroyAllWindows()